# LightGBM の実装

In [1]:
# ====================================================
# Library
# ====================================================
import os
import gc
import datetime
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import pandas as pd
from pathlib import Path
import pickle

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import lightgbm as lgb
from catboost import CatBoostClassifier
import feature_engineering as fe

In [2]:
!mkdir oof
!mkdir models

mkdir: oof: File exists
mkdir: models: File exists


In [3]:
train = pd.read_csv("train.csv")
card = pd.read_csv("card.csv")
test = pd.read_csv("test.csv")
user = pd.read_csv("user.csv")
submission_df = pd.read_csv("sample_submit.csv")
all_df = fe.apply_feature_engineering(train, test, card, user)

# 最後に、訓練データとテストデータに再度分割します
train_df = all_df[all_df['is_train'] == 1]
test_df = all_df[all_df['is_train'] == 0]

In [4]:
# ====================================================
# Configurations
# ====================================================
class CFG:
    DATA_PATH = Path('.')
    OOF_DATA_PATH = Path('./oof')
    MODEL_DATA_PATH = Path('./models')
    # METHOD_LIST = ['catboost', 'lightgbm']
    METHOD_LIST = ['lightgbm']
    seed = random.randint(0, 100)
    n_folds = 5
    target_col = 'is_fraud?'
    USE_PLAYER_FEATURES = False
    num_boost_round = 50000
    early_stopping_round = 500
    verbose = 2000
    boosting_type = 'gbdt'
    lgb_params = {
        'objective': 'binary',
        'boosting': boosting_type,
        'n_jobs': -1,
        'seed': seed,
        'force_col_wise': 'true',
        'num_boost_round': num_boost_round
    }
    cat_params = {
        'iterations': num_boost_round,
        'random_seed': seed,
    }

In [5]:
# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(CFG.seed)

In [6]:
def lightgbm_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame):
    # Apply the encoding map to the relevant columns
    categorical_cols = x_train.select_dtypes(include=['object']).columns.tolist()

    # Convert categorical columns to 'category' dtype
    for col in categorical_cols:
        x_train[col] = x_train[col].astype('category')
        x_valid[col] = x_valid[col].astype('category')

    # Create LightGBM datasets
    train_data = lgb.Dataset(x_train, label=y_train, categorical_feature=categorical_cols)
    valid_data = lgb.Dataset(x_valid, label=y_valid, categorical_feature=categorical_cols)

    # Train model
    model = lgb.train(
        params=CFG.lgb_params,
        train_set=train_data,
        valid_sets=[valid_data],
        callbacks=[lgb.early_stopping(CFG.early_stopping_round, verbose=True), lgb.log_evaluation(CFG.verbose)]
        )

    # Predict validation
    valid_pred = model.predict(x_valid)
    return model, valid_pred

def catboost_training(x_train: pd.DataFrame, y_train: pd.DataFrame, x_valid: pd.DataFrame, y_valid: pd.DataFrame, categorical_features: list):
    # Define model
    model = CatBoostClassifier(**CFG.cat_params, cat_features=categorical_features)

    # Fit model
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], early_stopping_rounds=CFG.early_stopping_round, verbose=CFG.verbose)

    # Predict validation
    valid_pred = model.predict(x_valid)

    return model, valid_pred

def gradient_boosting_model_cv_training(method: str, train_df: pd.DataFrame, features: list, categorical_features: list):
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train_df))
    oof_fold = np.zeros(len(train_df))
    kfold = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
    for fold, (train_index, valid_index) in enumerate(kfold.split(train_df[features], train_df[CFG.target_col])):
        print('-'*50)
        print(f'{method} training fold {fold + 1}')
        x_train = train_df[features].iloc[train_index]
        y_train = train_df[CFG.target_col].iloc[train_index]
        x_valid = train_df[features].iloc[valid_index]
        y_valid = train_df[CFG.target_col].iloc[valid_index]
        if method == 'lightgbm':
            model, valid_pred = lightgbm_training(x_train, y_train, x_valid, y_valid)
        if method == 'catboost':
            model, valid_pred = catboost_training(x_train, y_train, x_valid, y_valid, categorical_features)

        # Save best model
        pickle.dump(model, open(CFG.MODEL_DATA_PATH / f'{method}_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'wb'))

        # Add to out of folds array
        oof_predictions[valid_index] = valid_pred
        oof_fold[valid_index] = fold + 1
        del x_train, x_valid, y_train, y_valid, model, valid_pred
        gc.collect()

    # Compute out of folds metric
    binary_predictions = [1 if p>0.5 else 0 for p in oof_predictions]
    score = f1_score(train_df[CFG.target_col], binary_predictions)
    print(f'{method} our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    # oof_df = pd.DataFrame({'id': train_df['id'], CFG.target_col: train_df[CFG.target_col], f'{method}_prediction': oof_predictions, 'fold': oof_fold})
    # oof_df.to_csv(CFG.MODEL_DATA_PATH / f'oof_{method}_seed{CFG.seed}_ver{CFG.boosting_type}.csv', index = False)

In [7]:
train_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 471283 entries, 0 to 471282
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   index                        471283 non-null  int64  
 1   amount                       471283 non-null  float64
 2   is_fraud?                    471283 non-null  float64
 3   merchant_id                  471283 non-null  int64  
 4   mcc                          471283 non-null  int64  
 5   is_train                     471283 non-null  int64  
 6   cards_issued                 471283 non-null  int64  
 7   credit_limit                 471283 non-null  float64
 8   year_pin_last_changed        471283 non-null  int64  
 9   current_age                  471283 non-null  int64  
 10  retirement_age               471283 non-null  int64  
 11  birth_year                   471283 non-null  int64  
 12  zipcode                      471283 non-null  int64  
 13  lati

In [8]:
# SMOTEによるアップサンプリング
train_df = fe.smote_optimize_and_apply(train_df, CFG.target_col, random_state=CFG.seed)

In [9]:
# 'Id'や'Target'といった特定のカラムを除外した全てのカラムを特徴量とする場合
categorical_features = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
features = train_df.columns.drop(['index', 'is_fraud?'])

In [10]:
for method in CFG.METHOD_LIST:
    gradient_boosting_model_cv_training(method, train_df, features, categorical_features)

--------------------------------------------------
lightgbm training fold 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 175461, number of negative: 350921
[LightGBM] [Info] Total Bins 5649
[LightGBM] [Info] Number of data points in the train set: 526382, number of used features: 40
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333334 -> initscore=-0.693144
[LightGBM] [Info] Start training from score -0.693144
Training until validation scores don't improve for 500 rounds
[2000]	valid_0's binary_logloss: 0.116617
[4000]	valid_0's binary_logloss: 0.114447
Early stopping, best iteration is:
[3911]	valid_0's binary_logloss: 0.11434
--------------------------------------------------
lightgbm training fold 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 175461, numbe

In [11]:
def lightgbm_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(CFG.MODEL_DATA_PATH / f'lightgbm_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'rb'))
        categorical_cols = x_test.select_dtypes(include=['object', 'category']).columns
        for col in categorical_cols:
            x_test[col] = x_test[col].astype('category')
        test_pred += model.predict(x_test)
    return test_pred / CFG.n_folds

def catboost_inference(x_test: pd.DataFrame):
    test_pred = np.zeros(len(x_test))
    for fold in range(CFG.n_folds):
        model = pickle.load(open(CFG.MODEL_DATA_PATH / f'catboost_fold{fold + 1}_seed{CFG.seed}_ver{CFG.boosting_type}.pkl', 'rb'))
        test_pred += model.predict(x_test)
    return test_pred / CFG.n_folds

def gradient_boosting_model_inference(method: str, test_df: pd.DataFrame, features: list):
    x_test = test_df[features]
    if method == 'lightgbm':
        test_pred = lightgbm_inference(x_test)
    if method == 'catboost':
        test_pred = catboost_inference(x_test)
    return test_pred


for method in CFG.METHOD_LIST:
    test_df[f'{method}_pred'] = gradient_boosting_model_inference(method, test_df, features)


# # アンサンブルの結果を保存
# test_df['final_pred'] = np.round(0.5 * test_df['lightgbm_pred'] + 0.5 * test_df['catboost_pred']).astype(int)
# test_df[['index','final_pred']].to_csv(f'submissions/EMS_{CFG.n_folds}folds_submission_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}_{CFG.seed}.csv', index=False, header=False)

# LGBのみの結果を保存
test_df['final_pred'] = np.round(test_df['lightgbm_pred']).astype(int)
test_df[['index','final_pred']].to_csv(f'submissions/OnlyLGB_{CFG.n_folds}folds_submission_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}_{CFG.seed}.csv', index=False, header=False)

# # Catのみの結果を保存
# test_df['final_pred'] = np.round(test_df['catboost_pred']).astype(int)
# test_df[['index','final_pred']].to_csv(f'submissions/OnlyCat_{CFG.n_folds}folds_submission_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}_{CFG.seed}.csv', index=False, header=False)

In [12]:
# ターゲット変数が 'target_column' だと仮定
value_counts = train_df[CFG.target_col].value_counts()
print(value_counts)

# パーセンテージで表示する場合
# value_counts_percent = train_df['target_column'].value_counts(normalize=True) * 100
# print(value_counts_percent)

is_fraud?
0.0    438652
1.0    219326
Name: count, dtype: int64
